# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21096838


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/31 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/31 [00:07<01:52,  3.88s/it]

Rendering models:  10%|▉         | 3/31 [00:08<01:23,  2.99s/it]

Rendering models:  16%|█▌        | 5/31 [00:09<00:57,  2.20s/it]

Rendering models:  19%|█▉        | 6/31 [00:10<00:44,  1.79s/it]

...

Rendering models:  87%|████████▋ | 27/31 [00:24<00:02,  1.80it/s]

Rendering models:  90%|█████████ | 28/31 [00:25<00:01,  1.77it/s]

Rendering models:  94%|█████████▎| 29/31 [00:26<00:01,  1.53it/s]

Rendering models:  97%|█████████▋| 30/31 [00:26<00:00,  1.58it/s]

not-logged-in-3b0b929f4475fd276561    0.000998
not-logged-in-3b0b929f4475fd276561    0.100610
Jnursssmith5263                       0.001065
Alexander00                           0.000521
Quarkgc                               0.000391
not-logged-in-42aeec5653762437f6b7    0.000784
not-logged-in-c6917d9b19f41e6c2995    0.013404
not-logged-in-c6917d9b19f41e6c2995    0.054942
not-logged-in-9a813f8c400d5fedde5d    0.007564
not-logged-in-22877e3a7af4aafb750a    0.002652
not-logged-in-923f38b394a04ae3a577    0.016890
not-logged-in-314d48142a3a122ba666    0.000883
pangeli5                              0.000439
EM4893                                0.000487
mjcb4                                 0.000567
acapirala                             0.019969
not-logged-in-3ae935e3afaee843f530    0.002716
Lavadude                              0.011666
not-logged-in-e54074876180f935ede3    0.002133
equidad1                              0.003197
not-logged-in-295f3577632e98f73ca2    0.914104
MrMinecraft  

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())